# Libraries and Tools

In [1]:
from utils import *
from student_utils import *
import gurobipy as grb
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Graph Class

In [2]:
"""
A class that performs data preprocessing to gather important data and to provide useful functions
"""
class Graph:
    def __init__(self, input_file):
        parsed_data = data_parser(read_file(input_file))
        self.data = {}
        self.input_file = input_file
        self.number_of_locations = parsed_data[0] 
        self.number_of_houses = parsed_data[1] 
        self.list_of_locations = parsed_data[2]  
        self.list_of_houses = parsed_data[3] 
        self.starting_location = parsed_data[4] 
        self.adjacency_matrix = parsed_data[5] 
        self.G, message = adjacency_matrix_to_graph(self.adjacency_matrix)
        if message:
            print(message)
        else:
            print("Successful creation of Graph instance")
        
    """
    Returns a NumPy matrix of shortest path distances between every pair of nodes using the Floyd-Warshall algorithm. 
    If there is no path between to nodes the corresponding matrix entry will be Inf.
    """
    def get_fw_matrix(self):
        return nx.floyd_warshall_numpy(self.G)

In [3]:
graph = Graph("inputs/practice.in")

Successful creation of Graph instance


# Integer Linear Programming

In [4]:
model = grb.Model()

Academic license - for non-commercial use only


In [5]:
"""
Arrangement of Drop-Offs Matrix
"""
ARRANGEMENTS = []
for r in range(graph.number_of_locations):
    ROW = []
    for c in range(graph.number_of_houses + 2):
        matrix_element_name = "A_" + str(r) + "_" + str(c)
        ROW.append(model.addVar(vtype=grb.GRB.BINARY, name=matrix_element_name))
        model.update()
    ARRANGEMENTS.append(ROW)
ARRANGEMENTS = np.array(ARRANGEMENTS)

In [6]:
"""
Arrangement of Drop-Offs Constraints
"""
# Get the index number of where Soda is
soda = graph.list_of_locations.index(graph.starting_location)
# Check that we start at Soda
model.addConstr(ARRANGEMENTS[soda][0] == 1)
# Check that we end at Soda
model.addConstr(ARRANGEMENTS[soda][graph.number_of_houses + 1] == 1)
# Check that each column of ARRANGEMENTS sums up to 1
for c in range(len(ARRANGEMENTS[0])):
    model.addConstr(grb.quicksum(ARRANGEMENTS[:, c]) == 1)

In [7]:
"""
TA Walking Matrix
"""
WALKING = []
for r in range(graph.number_of_locations):
    ROW = []
    for c in range(graph.number_of_locations):
        matrix_element_name = "W_" + str(r) + "_" + str(c)
        ROW.append(model.addVar(vtype=grb.GRB.BINARY, name=matrix_element_name))
        model.update()
    WALKING.append(ROW)
WALKING = np.array(WALKING)

In [8]:
"""
TA Walking Constraints
"""
H = (np.array(convert_locations_to_indices(graph.list_of_locations, graph.list_of_houses)) != None).astype(int)

# Check that each column i of WALKING sums up to H[i]
for i in range(len(WALKING[0])):
    model.addConstr(grb.quicksum(WALKING[:, i]) == H[i])

In [9]:
"""
Driving Cost Function
"""
DISTANCES = graph.get_fw_matrix()
driving_cost_function = []
for c in range(graph.number_of_houses + 1):
    summation = []
    for i in range(graph.number_of_locations):
        for j in range(graph.number_of_locations):
            summation.append(
                grb.QuadExpr(ARRANGEMENTS[i][c] * DISTANCES.item((i, j)) * ARRANGEMENTS[j][c + 1])
            )
            model.update()
    driving_cost_function.append(grb.quicksum(summation))

In [10]:
"""
Walking Cost Function
"""
walking_cost_function = []
for row in range(graph.number_of_locations):
    for col in range(graph.number_of_locations):
        walking_cost_function.append(grb.LinExpr(WALKING[row][col] * DISTANCES.item((row, col))))
        model.update()

In [11]:
"""
Set Objective Function
"""
cost_function = driving_cost_function + walking_cost_function
model.setObjective(grb.quicksum(cost_function), grb.GRB.MINIMIZE)

In [12]:
"""
Minimize Objective Function
"""
model.optimize()

for v in model.getVars():
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % model.objVal)

Optimize a model with 65 rows, 2835 columns and 2837 nonzeros
Model has 33660 quadratic objective terms
Variable types: 0 continuous, 2835 integer (2835 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  QObjective range [2e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 290.5000000
Presolve removed 49 rows and 2115 columns
Presolve time: 0.12s
Presolved: 16 rows, 720 columns, 720 nonzeros
Presolved model has 30420 quadratic objective terms
Found heuristic solution: objective 49.8000000
Variable types: 0 continuous, 720 integer (720 binary)

Root relaxation: objective -1.126272e+04, 255 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -11262.723    0  206   49.80000    0.00000   100%     -    0s
H    0     0                      1

W_0_39 0
W_0_40 0
W_0_41 0
W_0_42 0
W_0_43 0
W_0_44 0
W_1_0 0
W_1_1 0
W_1_2 0
W_1_3 0
W_1_4 0
W_1_5 0
W_1_6 0
W_1_7 0
W_1_8 0
W_1_9 0
W_1_10 0
W_1_11 0
W_1_12 0
W_1_13 0
W_1_14 0
W_1_15 0
W_1_16 0
W_1_17 0
W_1_18 0
W_1_19 0
W_1_20 0
W_1_21 0
W_1_22 0
W_1_23 0
W_1_24 0
W_1_25 0
W_1_26 0
W_1_27 0
W_1_28 0
W_1_29 0
W_1_30 0
W_1_31 0
W_1_32 0
W_1_33 0
W_1_34 0
W_1_35 0
W_1_36 0
W_1_37 0
W_1_38 0
W_1_39 0
W_1_40 0
W_1_41 0
W_1_42 0
W_1_43 0
W_1_44 0
W_2_0 0
W_2_1 0
W_2_2 0
W_2_3 0
W_2_4 0
W_2_5 0
W_2_6 0
W_2_7 0
W_2_8 0
W_2_9 0
W_2_10 0
W_2_11 0
W_2_12 0
W_2_13 0
W_2_14 0
W_2_15 0
W_2_16 0
W_2_17 0
W_2_18 0
W_2_19 0
W_2_20 0
W_2_21 0
W_2_22 0
W_2_23 0
W_2_24 0
W_2_25 0
W_2_26 0
W_2_27 0
W_2_28 0
W_2_29 0
W_2_30 0
W_2_31 0
W_2_32 0
W_2_33 0
W_2_34 0
W_2_35 0
W_2_36 0
W_2_37 0
W_2_38 0
W_2_39 0
W_2_40 0
W_2_41 0
W_2_42 0
W_2_43 0
W_2_44 0
W_3_0 0
W_3_1 0
W_3_2 0
W_3_3 0
W_3_4 0
W_3_5 0
W_3_6 0
W_3_7 0
W_3_8 0
W_3_9 0
W_3_10 0
W_3_11 0
W_3_12 0
W_3_13 0
W_3_14 0
W_3_15 0
W_3_16 0
W_3_17 0
W_3_

W_23_4 0
W_23_5 0
W_23_6 0
W_23_7 0
W_23_8 0
W_23_9 0
W_23_10 0
W_23_11 0
W_23_12 0
W_23_13 0
W_23_14 0
W_23_15 0
W_23_16 0
W_23_17 0
W_23_18 0
W_23_19 0
W_23_20 0
W_23_21 0
W_23_22 0
W_23_23 1
W_23_24 0
W_23_25 0
W_23_26 0
W_23_27 0
W_23_28 0
W_23_29 0
W_23_30 0
W_23_31 0
W_23_32 0
W_23_33 0
W_23_34 0
W_23_35 0
W_23_36 0
W_23_37 0
W_23_38 0
W_23_39 0
W_23_40 0
W_23_41 0
W_23_42 0
W_23_43 0
W_23_44 0
W_24_0 0
W_24_1 0
W_24_2 0
W_24_3 0
W_24_4 0
W_24_5 0
W_24_6 0
W_24_7 0
W_24_8 0
W_24_9 0
W_24_10 0
W_24_11 0
W_24_12 0
W_24_13 0
W_24_14 0
W_24_15 0
W_24_16 0
W_24_17 0
W_24_18 0
W_24_19 0
W_24_20 0
W_24_21 0
W_24_22 0
W_24_23 0
W_24_24 1
W_24_25 0
W_24_26 0
W_24_27 0
W_24_28 0
W_24_29 0
W_24_30 0
W_24_31 0
W_24_32 0
W_24_33 0
W_24_34 0
W_24_35 0
W_24_36 0
W_24_37 0
W_24_38 0
W_24_39 0
W_24_40 0
W_24_41 0
W_24_42 0
W_24_43 0
W_24_44 0
W_25_0 0
W_25_1 0
W_25_2 0
W_25_3 0
W_25_4 0
W_25_5 0
W_25_6 0
W_25_7 0
W_25_8 0
W_25_9 0
W_25_10 0
W_25_11 0
W_25_12 0
W_25_13 0
W_25_14 0
W_25_15 0
W_25_1